In [103]:
import pandas as pd
import numpy as np
import os
import pickle
import lightgbm as lgb
from sklearn.model_selection import train_test_split,GridSearchCV,RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from imblearn.over_sampling import SMOTE
from math import log
from imblearn.combine import SMOTETomek
from imblearn.under_sampling import TomekLinks

In [104]:
pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns',None)
pd.set_option('display.width',None)
pd.set_option('display.max_colwidth',None)

In [105]:
### Loading the required files

In [106]:
pickle_in = open("Transformed_Data_.pickle","rb")
Final_Data = pickle.load(pickle_in)
pickle_in.close()

In [107]:
### Split data into Train, Test and Validation

In [108]:
x = Final_Data.drop('SEL_FLAG2',axis=1)
y = pd.DataFrame(Final_Data['SEL_FLAG2'])
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.1,shuffle=True,random_state=0)
x_train,x_val,y_train,y_val = train_test_split(x_train,y_train,test_size=0.22222,shuffle=True,random_state=0)

In [109]:
y_train =y_train.astype({'SEL_FLAG2':'uint8'})
y_test = y_test.astype({'SEL_FLAG2':'uint8'})
y_val = y_val.astype({'SEL_FLAG2':'uint8'})

In [110]:
## Checking the Selection %

In [111]:
print(y_train['SEL_FLAG2'].value_counts()/len(y_train)*100) #7.83
print(y_test['SEL_FLAG2'].value_counts()/len(y_test)*100)   #8.0
print(y_val['SEL_FLAG2'].value_counts()/len(y_val)*100)   #8.0

0    92.163504
1     7.836496
Name: SEL_FLAG2, dtype: float64
0    92.234439
1     7.765561
Name: SEL_FLAG2, dtype: float64
0    91.991262
1     8.008738
Name: SEL_FLAG2, dtype: float64


In [112]:
####Data PreProcessing (numeric columns of train_data)

##Separating out Numeric columns from train data

In [113]:
train_num = x_train.select_dtypes(include='number')

In [114]:
train_num.dtypes

DIAG_CNT          Int64
HHPA_1          Float64
HHPA_2          Float64
HHPA_3          Float64
HHPA_4          Float64
HHPA_5            Int64
HHPA_6          Float64
HHPA_7          Float64
HHPA_8            Int64
HHPA_9          Float64
HHPA_10         Float64
HHPA_11         Float64
HHPA_12         Float64
HHPA_13         Float64
HHPA_14         Float64
HHPA_15         Float64
HHPA_16         Float64
HHPA_17         Float64
HHPA_18         Float64
HHPA_19         Float64
HHPA_20         Float64
HHPA_21         Float64
HHPA_22         Float64
HHPA_23         Float64
HHPA_24         Float64
HHPA_25         Float64
HHPA_26         Float64
HHPA_27         Float64
HHPA_28         Float64
HHPA_29         Float64
HHPA_30         Float64
HHPA_31         Float64
HHPA_OTHERS     Float64
HHSUC_1         Float64
HHSUC_2         Float64
HHSUC_3           Int64
HHSUC_4           Int64
HHSUC_5           Int64
HHSUC_6           Int64
HHSUC_7           Int64
HHSUC_8           Int64
HHSUC_9         

In [115]:
##PreProcessing numeric data

In [116]:
sc = StandardScaler()
train_num_process = pd.DataFrame(sc.fit_transform(train_num))
train_num_process.columns = train_num.columns
train_num_process.index = train_num.index

In [117]:
pickle_out = open('Scaler.pickle','wb')
pickle.dump(sc,pickle_out)
pickle_out.close()

In [118]:
### Combining the processed data to train_data

In [119]:
col_drop = train_num.columns
x_train = x_train.drop(col_drop,axis=1)

In [120]:
## Combining the PreProcessed data to train_data

In [121]:
x_train = pd.concat([x_train,train_num_process],axis=1)

In [122]:
x_train = x_train.astype({'REV_0551':'uint8','REV_0421':'uint8','REV_0023':'uint8','REV_0431':'uint8','REV_0270':'uint8',
'REV_0623':'uint8','REV_0571':'uint8','REV_0561':'uint8','REV_0441':'uint8',
'REV_0550':'uint8','REV_0420':'uint8','REV_OTHERS':'uint8',
'NTWK_ID_5241H3I':'uint8','NTWK_ID_0000H32':'uint8',
'NTWK_ID_5367H3I':'uint8',
'NTWK_ID_7030H3I':'uint8',
'NTWK_ID_0000H3I':'uint8',
'NTWK_ID_7037H32':'uint8',
'NTWK_ID_5366H32':'uint8',
'NTWK_ID_OTHERS':'uint8',
'LINE_CAT_CD_3030':'uint8',
'LINE_CAT_CD_3100':'uint8',
'LINE_CAT_CD_9900':'uint8',
'LINE_CAT_CD_OTHERS':'uint8'})

In [123]:
## performing one-hot-encoding on categorical features

In [124]:
ctgrl_col = pd.DataFrame(x_train[['ADMIT_SRC_CD','BILL_TYPE_CD','CAUSE_CD','MAPPING_LEVEL_2','SRC_PAR_CD']])
ctgrl_cols = pd.get_dummies(ctgrl_col)

x_train = x_train.drop(ctgrl_col.columns,axis=1)
x_train = pd.concat([x_train,ctgrl_cols],axis=1)

In [125]:
y_train.value_counts(normalize=True)*100

SEL_FLAG2
0            92.163504
1             7.836496
dtype: float64

In [126]:
###Val Data

In [127]:
val_num = x_val.select_dtypes(include='number')

In [128]:
val_num_process = pd.DataFrame(sc.transform(val_num))
val_num_process.columns = val_num.columns
val_num_process.index = val_num.index

In [129]:
col_drop = val_num.columns
x_val = x_val.drop(col_drop,axis=1)

In [130]:
x_val = pd.concat([x_val,val_num_process],axis=1)

In [131]:
x_val = x_val.astype({'REV_0551':'uint8','REV_0421':'uint8','REV_0023':'uint8','REV_0431':'uint8','REV_0270':'uint8',
'REV_0623':'uint8','REV_0571':'uint8','REV_0561':'uint8','REV_0441':'uint8',
'REV_0550':'uint8','REV_0420':'uint8','REV_OTHERS':'uint8',
'NTWK_ID_5241H3I':'uint8','NTWK_ID_0000H32':'uint8',
'NTWK_ID_5367H3I':'uint8',
'NTWK_ID_7030H3I':'uint8',
'NTWK_ID_0000H3I':'uint8',
'NTWK_ID_7037H32':'uint8',
'NTWK_ID_5366H32':'uint8',
'NTWK_ID_OTHERS':'uint8',
'LINE_CAT_CD_3030':'uint8',
'LINE_CAT_CD_3100':'uint8',
'LINE_CAT_CD_9900':'uint8',
'LINE_CAT_CD_OTHERS':'uint8'})

In [132]:
ctgrl_col = pd.DataFrame(x_val[['ADMIT_SRC_CD','BILL_TYPE_CD','CAUSE_CD','MAPPING_LEVEL_2','SRC_PAR_CD']])
ctgrl_cols = pd.get_dummies(ctgrl_col)

x_val = x_val.drop(ctgrl_col.columns,axis=1)
x_val = pd.concat([x_val,ctgrl_cols],axis=1)

In [133]:
##model building

In [134]:
model = lgb.LGBMClassifier()

In [135]:
parameters = {'objective':['binary'],
              "max_depth": [7,9,11],
              "learning_rate" : [0.1,0.001,0.003],
              "min_data_in_leaf": [100,200,300],
              "n_estimators": [500],
              'metric': ['binary_logloss'],
              'is_unbalance': [True],
              'boosting': ['dart'],
              'feature_fraction': [0.6,0.7,0.8],
              'bagging_fraction': [0.5],
              'bagging_freq': [20,30,40],
              'verbose': [-1]
             }

In [136]:
grid_search = RandomizedSearchCV(model,parameters, cv = 15, 
                                   verbose=True)

grid_search.fit(x_train.values, y_train.values.ravel())

Fitting 15 folds for each of 10 candidates, totalling 150 fits
[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] feature_fraction is set=0.7, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] bagging_fraction is set=0.5, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5
[LightGBM] [Warning] bagging_freq is set=20, subsample_freq=0 will be ignored. Current value: bagging_freq=20
[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] feature_fraction is set=0.7, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=10

[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] feature_fraction is set=0.7, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] bagging_fraction is set=0.5, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5
[LightGBM] [Warning] bagging_freq is set=30, subsample_freq=0 will be ignored. Current value: bagging_freq=30
[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] feature_fraction is set=0.7, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] bagging_fraction is set=0.5, subsample=1

[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] bagging_fraction is set=0.5, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5
[LightGBM] [Warning] bagging_freq is set=30, subsample_freq=0 will be ignored. Current value: bagging_freq=30
[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] bagging_fraction is set=0.5, subsample=1

[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] bagging_fraction is set=0.5, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5
[LightGBM] [Warning] bagging_freq is set=20, subsample_freq=0 will be ignored. Current value: bagging_freq=20
[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] bagging_fraction is set=0.5, subsample=1

[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] min_data_in_leaf is set=300, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=300
[LightGBM] [Warning] bagging_fraction is set=0.5, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5
[LightGBM] [Warning] bagging_freq is set=20, subsample_freq=0 will be ignored. Current value: bagging_freq=20
[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] min_data_in_leaf is set=300, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=300
[LightGBM] [Warning] bagging_fraction is set=0.5, subsample=1

...

[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] min_data_in_leaf is set=200, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=200
[LightGBM] [Warning] bagging_fraction is set=0.5, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5
[LightGBM] [Warning] bagging_freq is set=20, subsample_freq=0 will be ignored. Current value: bagging_freq=20
[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] min_data_in_leaf is set=200, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=200
[LightGBM] [Warning] bagging_fraction is set=0.5, subsample=1

[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] feature_fraction is set=0.6, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6
[LightGBM] [Warning] min_data_in_leaf is set=300, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=300
[LightGBM] [Warning] bagging_fraction is set=0.5, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5
[LightGBM] [Warning] bagging_freq is set=40, subsample_freq=0 will be ignored. Current value: bagging_freq=40
[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] feature_fraction is set=0.6, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6
[LightGBM] [Warning] min_data_in_leaf is set=300, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=300
[LightGBM] [Warning] bagging_fraction is set=0.5, subsample=1

[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] feature_fraction is set=0.6, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] bagging_fraction is set=0.5, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5
[LightGBM] [Warning] bagging_freq is set=40, subsample_freq=0 will be ignored. Current value: bagging_freq=40
[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] feature_fraction is set=0.6, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] bagging_fraction is set=0.5, subsample=1

[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] min_data_in_leaf is set=300, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=300
[LightGBM] [Warning] bagging_fraction is set=0.5, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5
[LightGBM] [Warning] bagging_freq is set=20, subsample_freq=0 will be ignored. Current value: bagging_freq=20


RandomizedSearchCV(cv=15, estimator=LGBMClassifier(),
                   param_distributions={'bagging_fraction': [0.5],
                                        'bagging_freq': [20, 30, 40],
                                        'boosting': ['gbdt'],
                                        'feature_fraction': [0.6, 0.7, 0.8],
                                        'is_unbalance': [True],
                                        'learning_rate': [0.1, 0.001, 0.003],
                                        'max_depth': [7, 9, 11, 13],
                                        'metric': ['binary_logloss'],
                                        'min_data_in_leaf': [100, 200, 300],
                                        'n_estimators': [500],
                                        'objective': ['binary'],
                                        'verbose': [-1]},
                   verbose=True)

In [137]:
grid_search.best_estimator_

LGBMClassifier(bagging_fraction=0.5, bagging_freq=20, boosting='gbdt',
               feature_fraction=0.8, is_unbalance=True, learning_rate=0.001,
               max_depth=9, metric='binary_logloss', min_data_in_leaf=300,
               n_estimators=500, objective='binary', verbose=-1)

In [138]:
grid_search.best_score_

0.9216350400430253

In [139]:
model = lgb.LGBMClassifier(bagging_fraction=0.5, bagging_freq=20, boosting='gbdt',
               feature_fraction=0.8, is_unbalance=True, learning_rate=0.001,
               max_depth=9, metric='binary_logloss', min_data_in_leaf=300,
               n_estimators=500, objective='binary', verbose=-1)
model.fit(x_train.values,y_train.values.ravel())

LGBMClassifier(bagging_fraction=0.5, bagging_freq=20, boosting='gbdt',
               feature_fraction=0.8, is_unbalance=True, learning_rate=0.001,
               max_depth=9, metric='binary_logloss', min_data_in_leaf=300,
               n_estimators=500, objective='binary', verbose=-1)

In [140]:
pickle_out = open("Model_LGBM.pickle","wb")
pickle.dump(model,pickle_out)
pickle_out.close()

In [141]:
####################################################
####################################################
####################################################

### Model Building

In [142]:
#RF_v1 = RandomForestClassifier()

In [143]:
#data = {
#    'n_estimators':[500],
#    'criterion':['gini','entropy'],
#    'max_depth':[7,10,13],
#    'min_samples_split':[500,700,1000],
#    'max_features':['auto']
#}

In [144]:
#from sklearn.model_selection import RandomizedSearchCV
#gscv = RandomizedSearchCV(RF_v1,data,verbose=True,cv=15)
#gscv.fit(x_train.values,y_train.values.ravel())

In [145]:
#gscv.best_params_

In [146]:
#gscv.best_score_

In [147]:
#RF_v1 = RandomForestClassifier(n_estimators= 400,
# min_samples_split= 100,
# min_samples_leaf= 200,
# max_features= 'auto',
# max_depth= 15,
# criterion= 'gini')
#RF_v1.fit(x_train.values,y_train.values.ravel())

In [148]:
## Saving the model

In [149]:
#pickle_out = open("Model_RF.pickle","wb")
#pickle.dump(RF_v1,pickle_out)
#pickle_out.close()

In [150]:
#importances = RF_v1.feature_importances_

In [151]:
#feat = pd.DataFrame(RF_v1.feature_importances_,index=x_sm.columns,columns=['importance']).sort_values('importance',ascending=False)

In [152]:
#feat.to_csv('U:/Propensity_Model/Audit data/Final_Data/EDA/Model_Retrain5/Modelfeat.csv',index=True,header=True)

In [153]:
#####################################

####Data PreProcessing (numeric columns of test_data)

##Separating out Numeric columns from test data

In [154]:
test_num = x_test.select_dtypes(include='number')

In [155]:
##PreProcessing numeric data

In [156]:
test_num_process = pd.DataFrame(sc.transform(test_num))
test_num_process.columns = test_num.columns
test_num_process.index = test_num.index

In [157]:
###########
### Combining the processed data to test_data

In [158]:
col_drop = test_num.columns
x_test = x_test.drop(col_drop,axis=1)

In [159]:
## Combining the Preprocessed data to test_data

In [160]:
x_test = pd.concat([x_test,test_num_process],axis=1)

In [161]:
x_test = x_test.astype({'REV_0551':'uint8','REV_0421':'uint8','REV_0023':'uint8','REV_0431':'uint8','REV_0270':'uint8',
'REV_0623':'uint8','REV_0571':'uint8','REV_0561':'uint8','REV_0441':'uint8',
'REV_0550':'uint8','REV_0420':'uint8','REV_OTHERS':'uint8',
'NTWK_ID_5241H3I':'uint8','NTWK_ID_0000H32':'uint8',
'NTWK_ID_5367H3I':'uint8',
'NTWK_ID_7030H3I':'uint8',
'NTWK_ID_0000H3I':'uint8',
'NTWK_ID_7037H32':'uint8',
'NTWK_ID_5366H32':'uint8',
'NTWK_ID_OTHERS':'uint8',
'LINE_CAT_CD_3030':'uint8',
'LINE_CAT_CD_3100':'uint8',
'LINE_CAT_CD_9900':'uint8',
'LINE_CAT_CD_OTHERS':'uint8'})

In [162]:
##one-hot-encoding on categorical features

In [163]:
ctgrl_col = pd.DataFrame(x_test[['ADMIT_SRC_CD','BILL_TYPE_CD','CAUSE_CD','MAPPING_LEVEL_2','SRC_PAR_CD']])
ctgrl_cols = pd.get_dummies(ctgrl_col)

x_test = x_test.drop(ctgrl_col.columns,axis=1)
x_test = pd.concat([x_test,ctgrl_cols],axis=1)

In [164]:
##################
### Predicting on the test data

In [165]:
##RF_v1

In [170]:
#y_pred = model.predict(x_train)
y_pred = (model.predict_proba(x_train)[:,1]>0.3).astype('uint8')
print(metrics.accuracy_score(y_train,y_pred))
print(metrics.confusion_matrix(y_train,y_pred))

print(metrics.classification_report(y_train,y_pred))

0.8178356544788993
[[110791  20636]
 [  5341   5834]]
              precision    recall  f1-score   support

           0       0.95      0.84      0.90    131427
           1       0.22      0.52      0.31     11175

    accuracy                           0.82    142602
   macro avg       0.59      0.68      0.60    142602
weighted avg       0.90      0.82      0.85    142602



In [171]:
#y_pred = model.predict(x_train)
y_pred = (model.predict_proba(x_val)[:,1]>0.3).astype('uint8')
print(metrics.accuracy_score(y_val,y_pred))
print(metrics.confusion_matrix(y_val,y_pred))

print(metrics.classification_report(y_val,y_pred))

0.817441032815453
[[31622  5858]
 [ 1580  1683]]
              precision    recall  f1-score   support

           0       0.95      0.84      0.89     37480
           1       0.22      0.52      0.31      3263

    accuracy                           0.82     40743
   macro avg       0.59      0.68      0.60     40743
weighted avg       0.89      0.82      0.85     40743



In [68]:
#y_pred = model.predict(x_test)
y_pred = (model.predict_proba(x_test)[:,1]>0.5).astype('uint8')
print(metrics.accuracy_score(y_test,y_pred))
print(metrics.confusion_matrix(y_test,y_pred))

print(metrics.classification_report(y_test,y_pred))

0.7618299626938936
[[14337  4453]
 [  399  1183]]
              precision    recall  f1-score   support

           0       0.97      0.76      0.86     18790
           1       0.21      0.75      0.33      1582

    accuracy                           0.76     20372
   macro avg       0.59      0.76      0.59     20372
weighted avg       0.91      0.76      0.81     20372

